# 数据处理概述

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r2.6.0/resource/_static/logo_notebook.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/r2.6.0/zh_cn/features/dataset/mindspore_overview.ipynb) [![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r2.6.0/resource/_static/logo_download_code.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/r2.6.0/zh_cn/features/dataset/mindspore_overview.py) [![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r2.6.0/resource/_static/logo_source.svg)](https://gitee.com/mindspore/docs/blob/r2.6.0/docs/mindspore/source_zh_cn/features/dataset/overview.ipynb)

MindSpore Dataset 提供两种数据处理能力：数据处理Pipeline模式和数据处理轻量化模式。

1. 数据处理Pipeline模式：提供基于C++ Runtime的并发数据处理流水线（Pipeline）能力。用户通过定义数据集加载、数据变换、数据批处理（Batch）等流程，实现数据集的高效加载、高效处理、高效Batch。并发度可调、缓存可调等能力，实现为NPU卡训练提供零Bottle Neck的训练数据。

2. 数据处理轻量化模式：支持用户使用数据变换操作（如：Resize、Crop、HWC2CHW等）进行单个样本的数据处理。

本章节后续重点讲述两种数据处理模式。

## 数据处理Pipeline模式

用户通过API定义的Dataset流水线，运行训练进程后Dataset会从数据集中循环加载数据 -> 处理 -> Batch -> 迭代器，最终用于训练。

![MindSpore Dataset Pipeline](https://www.mindspore.cn/docs/zh-CN/r2.6.0/_images/dataset_pipeline.png)

如上图所示，MindSpore Dataset模块使得用户很简便地定义数据预处理Pipeline，并以最高效（多进程/多线程）的方式处理数据集中样本，具体的步骤参考如下：

- 数据集加载（Dataset）：用户可以方便地使用Dataset类 ([标准格式数据集加载](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.loading.html#%E6%A0%87%E5%87%86%E6%A0%BC%E5%BC%8F%E6%95%B0%E6%8D%AE%E9%9B%86%E5%8A%A0%E8%BD%BD)、[视觉数据集](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.loading.html#%E8%A7%86%E8%A7%89%E6%95%B0%E6%8D%AE%E9%9B%86)、[文本数据集](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.loading.html#%E6%96%87%E6%9C%AC%E6%95%B0%E6%8D%AE%E9%9B%86)、[音频数据集](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.loading.html#%E9%9F%B3%E9%A2%91%E6%95%B0%E6%8D%AE%E9%9B%86)) 来加载已支持的数据集，或者通过 UDF Loader + [GeneratorDataset 自定义数据集](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.GeneratorDataset.html#mindspore.dataset.GeneratorDataset) 实现Python层自定义数据集的加载。加载类方法可以使用多种Sampler、数据分片、数据shuffle等功能；

- 数据集操作（filter/ skip）：用户通过数据集对象方法 [.shuffle](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/operation/mindspore.dataset.Dataset.shuffle.html#mindspore.dataset.Dataset.shuffle)、[.filter](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/operation/mindspore.dataset.Dataset.filter.html#mindspore.dataset.Dataset.filter)、[.skip](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/operation/mindspore.dataset.Dataset.skip.html#mindspore.dataset.Dataset.skip)、[.split](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/operation/mindspore.dataset.Dataset.split.html#mindspore.dataset.Dataset.split)、[.take](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/operation/mindspore.dataset.Dataset.take.html#mindspore.dataset.Dataset.take)等来实现数据集的进一步混洗、过滤、跳过、最多获取条数等操作；

- 数据集样本变换操作（map）：用户可以将数据变换操作 （[vision数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E8%A7%86%E8%A7%89) ， [nlp数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E6%96%87%E6%9C%AC) ， [audio数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E9%9F%B3%E9%A2%91) ） 添加到map操作中执行，数据预处理过程中可以定义多个map操作，用于执行不同变换操作。数据变换操作也可以是用户自定义变换的 PyFunc ；

- 批（batch）：用户在样本完成变换后，使用 [.batch](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/batch/mindspore.dataset.Dataset.batch.html#mindspore.dataset.Dataset.batch) 操作将多个样本组织成batch，也可以通过batch的参数 per_batch_map 来自定义batch逻辑；

- 迭代器（create_dict_iterator）：最后用户通过数据集对象方法 [.create_dict_iterator](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/iterator/mindspore.dataset.Dataset.create_dict_iterator.html#mindspore.dataset.Dataset.create_dict_iterator)、[.create_tuple_iterator](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/dataset_method/iterator/mindspore.dataset.Dataset.create_tuple_iterator.html#mindspore.dataset.Dataset.create_tuple_iterator) 来创建迭代器将预处理完成的数据循环输出。

### 数据集加载

下面主要介绍单个数据集加载、数据集组合、数据集切分、数据集保存等常用数据集加载方式。

#### 单个数据集加载

数据集加载类用于实现本地磁盘、OBS和共享存储上的训练数据集加载，将存储上的数据集Load至内存中。数据集加载接口如下：

| 数据集接口分类  | API列表  | 说明 |
|------------------------|----------------------------------------------------------|--------------------------------------------------------------|
| 标准格式数据集  | [MindDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.MindDataset.html#mindspore.dataset.MindDataset) 、 [TFRecordDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.TFRecordDataset.html#mindspore.dataset.TFRecordDataset) 、 [CSVDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.CSVDataset.html#mindspore.dataset.CSVDataset) 等 | 其中 MindDataset 依赖 MindSpore 数据格式， 详见： [格式转换](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/record.html) |
| 自定义数据集  | [GeneratorDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.GeneratorDataset.html#mindspore.dataset.GeneratorDataset) 、 [RandomDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.RandomDataset.html#mindspore.dataset.RandomDataset) 等 | 其中 GeneratorDataset 负责加载用户自定义DataLoader， 详见： [自定义数据集](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E9%9B%86) |
| 常用数据集  | [ImageFolderDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.ImageFolderDataset.html#mindspore.dataset.ImageFolderDataset) 、 [Cifar10Dataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.Cifar10Dataset.html#mindspore.dataset.Cifar10Dataset) 、 [IWSLT2017Dataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.IWSLT2017Dataset.html#mindspore.dataset.IWSLT2017Dataset) 、 [LJSpeechDataset](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/dataset/mindspore.dataset.LJSpeechDataset.html#mindspore.dataset.LJSpeechDataset) 等 | 用于常用的开源数据集 |

以上数据集加载（[示例](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E6%95%B0%E6%8D%AE%E9%9B%86%E5%8A%A0%E8%BD%BD)）中，可以配置不同的参数，以实现不同的加载效果，常用参数举例如下：

- `columns_list`：过滤数据集中指定的列，仅针对部分数据集接口。默认值：None，加载所有数据列。

- `num_parallel_workers`：配置数据集的读取并发数。默认值：8。

- 通过参数配置数据集的采样逻辑：

    - `shuffle`：开启混洗。默认值：True。

    - `num_shards` 和 `shard_id`：对数据集进行分片。默认值：None，不分片。

    - 其他更多的采样逻辑可以参考：[数据采样](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/sampler.html)。

#### 数据集组合

数据集组合可以将多个数据集以串联/并朕的方式组合起来，形成一个全新的dataset对象。

- 串联多个数据集

In [27]:
import mindspore.dataset as ds

ds.config.set_seed(1234)

data = [1, 2, 3]
dataset1 = ds.NumpySlicesDataset(data=data, column_names=["column_1"])

data = [4, 5, 6]
dataset2 = ds.NumpySlicesDataset(data=data, column_names=["column_1"])

dataset = dataset1.concat(dataset2)
for item in dataset.create_dict_iterator():
    print(item)

{'column_1': Tensor(shape=[], dtype=Int32, value= 3)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 2)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 1)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 6)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 5)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 4)}


- 并联多个数据集

In [28]:
import mindspore.dataset as ds

ds.config.set_seed(1234)

data = [1, 2, 3]
dataset1 = ds.NumpySlicesDataset(data=data, column_names=["column_1"])

data = [4, 5, 6]
dataset2 = ds.NumpySlicesDataset(data=data, column_names=["column_2"])

dataset = dataset1.zip(dataset2)
for item in dataset.create_dict_iterator():
    print(item)

{'column_1': Tensor(shape=[], dtype=Int32, value= 3), 'column_2': Tensor(shape=[], dtype=Int32, value= 6)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 2), 'column_2': Tensor(shape=[], dtype=Int32, value= 5)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 1), 'column_2': Tensor(shape=[], dtype=Int32, value= 4)}


#### 数据集切分

将数据集切分成训练数据集和验证数据集，分别用于训练过程和验证过程。

In [29]:
import mindspore.dataset as ds

data = [1, 2, 3, 4, 5, 6]
dataset = ds.NumpySlicesDataset(data=data, column_names=["column_1"], shuffle=False)

train_dataset, eval_dataset = dataset.split([4, 2])

print(">>>> train dataset >>>>")
for item in train_dataset.create_dict_iterator():
    print(item)

>>>> train dataset >>>>
{'column_1': Tensor(shape=[], dtype=Int32, value= 5)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 2)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 6)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 1)}


In [30]:
print(">>>> eval dataset >>>>")
for item in eval_dataset.create_dict_iterator():
    print(item)

>>>> eval dataset >>>>
{'column_1': Tensor(shape=[], dtype=Int32, value= 3)}
{'column_1': Tensor(shape=[], dtype=Int32, value= 4)}


#### 数据集保存

将数据集重新保存到MindRecord数据格式。

In [31]:
import os
import mindspore.dataset as ds

ds.config.set_seed(1234)

data = [1, 2, 3, 4, 5, 6]
dataset = ds.NumpySlicesDataset(data=data, column_names=["column_1"])
if os.path.exists("./train_dataset.mindrecord"):
    os.remove("./train_dataset.mindrecord")
if os.path.exists("./train_dataset.mindrecord.db"):
    os.remove("./train_dataset.mindrecord.db")
dataset.save("./train_dataset.mindrecord")

### 数据变换

#### 普通数据变换

用户可以使用多种数据变换操作：

- `.map(...)`：变换操作。
- `.filter(...)`：过滤操作。
- `.project(...)`：对多列进行排序和过滤。
- `.rename(...)` 对指定列重命名。
- `.shuffle(...)` 对数据进行缓存区大小的混洗。
- `.skip(...)` 跳过数据集的前 n 条。
- `.take(...)` 只读数据集的前 n 条样本。

下面重点说明 `.map(...)` 的使用方法：

- 在 `.map(...)` 中使用Dataset提供的数据变换操作

    Dataset提供了丰富的[数据变换操作](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#)，这些数据变换操作可以直接放在 `.map(...)` 中使用。具体使用方法参考 [map变换操作](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E5%86%85%E7%BD%AE%E6%95%B0%E6%8D%AE%E5%8F%98%E6%8D%A2%E6%93%8D%E4%BD%9C)。

- 在 `.map(...)` 中使用自定义数据变换操作

    Dataset也支持用户自定义的数据变换操作，仅需将用户自定义函数传递给 `.map(...)` 退可。具体使用方法参考：[自定义map变换操作](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%8F%98%E6%8D%A2%E6%93%8D%E4%BD%9C)。

- 在 `.map(...)` 中返回Dict数据结构数据

    Dataset也支持在用户自定义的数据变换操作中返回Dict数据结构，使得定义的数据变换更加灵活。具体使用方法参考：[自定义map变换操作处理字典对象](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/python_objects.html#%E8%87%AA%E5%AE%9A%E4%B9%89map%E5%A2%9E%E5%BC%BA%E6%93%8D%E4%BD%9C%E5%A4%84%E7%90%86%E5%AD%97%E5%85%B8%E5%AF%B9%E8%B1%A1)。

#### 自动数据增强

除了以上的普通数据变换，Dataset 还提供了一种自动数据变换方式，可以基于特定策略自动对图像进行数据变换处理。详细说明见：[自动数据增强](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/augment.html)。

### 数据batch

Dataset提供 `.batch(...)` 操作，可以很方便的将数据变换操作后的样本组织成batch。有两种使用方式：

1. 默认 `.batch(...)` 操作，将batch_size个样本组织成shape为 (batch_size, ...)的数据，详细用法请参考 [batch操作](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E6%95%B0%E6%8D%AEbatch)；

2. 自定义 `.batch(..., per_batch_map, ...)` 操作，支持用户将 [np.ndarray, nd.ndarray, ...] 多条数据按照自定义逻辑组织batch，详细用法请参考 [自定义batch操作](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/python_objects.html#batch%E6%93%8D%E4%BD%9C%E5%A4%84%E7%90%86%E5%AD%97%E5%85%B8%E5%AF%B9%E8%B1%A1)。

### 数据集迭代器

用户在定义完成 `数据集加载（xxDataset）-> 数据处理（.map）-> 数据batch（.batch）` Dataset流水线（Pipeline）后，可以通过创建迭代器方法 `.create_dict_iterator(...)` / `.create_tuple_iterator(...)` 循环将数据输出。具体的使用方法参考：[数据集迭代器](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/beginner/dataset.html#%E6%95%B0%E6%8D%AE%E9%9B%86%E8%BF%AD%E4%BB%A3%E5%99%A8)。

### 性能优化

#### 数据处理性能优化

针对数据处理Pipeline性能不足的场景，可以参考[数据处理性能优化](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/optimize.html)来进一步优化性能，以满足训练端到端性能要求。

#### 单节点数据缓存

另外，对于推理场景，为了追求极致的性能，可以使用 [单节点数据缓存](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/cache.html) 将数据集缓存于本地内存中，以加速数据集的读取和预处理。

## 数据处理轻量化模式

用户可以直接使用数据变换操作处理一条数据，返回值即是数据变换的结果。

数据变换操作（[vision数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E8%A7%86%E8%A7%89) ， [nlp数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E6%96%87%E6%9C%AC) ， [audio数据变换](https://www.mindspore.cn/docs/zh-CN/r2.6.0/api_python/mindspore.dataset.transforms.html#%E9%9F%B3%E9%A2%91)）可以像调用普通函数一样直接来使用。常见用法是：先初始化数据变换对象，然后调用数据变换操作方法，传入需要处理的数据，最后得到处理的结果。

In [32]:
from download import download
from PIL import Image
import mindspore.dataset.vision as vision

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/banana.jpg"
download(url, './banana.jpg', replace=True)


file_sizes: 100%|██████████████████████████| 17.1k/17.1k [00:00<00:00, 8.55MB/s]
Successfully downloaded file to ./banana.jpg


'./banana.jpg'

In [33]:

img_ori = Image.open("banana.jpg").convert("RGB")
print("Image.type: {}, Image.shape: {}".format(type(img_ori), img_ori.size))

Image.type: <class 'PIL.Image.Image'>, Image.shape: (356, 200)


In [34]:
# Apply Resize to input immediately
resize_op = vision.Resize(size=(320))
img = resize_op(img_ori)
print("Image.type: {}, Image.shape: {}".format(type(img), img.size))

Image.type: <class 'PIL.Image.Image'>, Image.shape: (569, 320)


更多的示例请参考：[轻量化数据处理](https://www.mindspore.cn/tutorials/zh-CN/r2.6.0/dataset/eager.html)。